<a href="https://colab.research.google.com/github/itsonlyaglitch/Code-en-commun/blob/main/Test%20model/keras_%2B_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://swatimeena989.medium.com/bert-text-classification-using-keras-903671e0207d

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import tensorflow.keras
from tqdm import tqdm
import pickle
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
import itertools
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
!pip install transformers
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

     |████████████████████████████████| 1.3MB 20.5MB/s 
     |████████████████████████████████| 2.9MB 53.0MB/s 
     |████████████████████████████████| 890kB 53.9MB/s 
     |████████████████████████████████| 1.1MB 48.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a073798b92fbda0f7db913a36e2c1e3b3ea5f398dacde52b52546e31220da646
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [4]:
!pip install PyDrive

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:

downloaded = drive.CreateFile({'id':"1JzRnzuYH40Bte2yrgkpJD408Sr9IuHRn"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.json')
df = pd.read_json('train.json')
df.head()

,Id,description,gender
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F
1,1,He is a member of the AICPA and WICPA. Brent ...,M
2,2,Dr. Aster has held teaching and research posi...,M
4,3,He runs a boutique design studio attending cl...,M
5,4,"He focuses on cloud security, identity and ac...",M


In [8]:
downloaded = drive.CreateFile({'id':"1gyxlEkzSBB3rJ6FYq6pIiOvVnK0u64wI"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test.json')
test_df = pd.read_json('test.json')
test_df.head()

,Id,description,gender
3,0,She currently works on CNN’s newest primetime...,F
6,1,Lavalette’s photographs have been shown widel...,M
11,2,Along with his academic and professional deve...,M
17,3,She obtained her Ph.D. in Islamic Studies at ...,F
18,4,She studies issues of women and Islam and has...,F


In [9]:
downloaded = drive.CreateFile({'id':"1pjtefNzFzGCOzWr7R3oNlzv9k4tMAkUK"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_label.csv')
df_label = pd.read_csv('train_label.csv')
df_label.head()

,Id,Category
0,0,19
1,1,9
2,2,19
3,3,24
4,4,24


In [10]:
downloaded = drive.CreateFile({'id':"16rXcCJWKPGeIZ9h4f3t-QoOShyoPcDaf"})   # replace the id with id of file you want to access
downloaded.GetContentFile('categories_string.csv')
categories_label = pd.read_csv('categories_string.csv')
categories_label.head()

,0,1
0,pastor,0
1,model,1
2,yoga_teacher,2
3,teacher,3
4,personal_trainer,4


In [ ]:
df = df.merge(df_label,on='Id')
df['job'] = categories_label.loc[df['Category'].values,'0'].values
df.head()

In [14]:
nltk.download('stopwords')
df['description'] = df['description'].map(preprocess_sentence)

In [15]:
df.head()

,Id,description,gender,Category,job
0,0,also ronald asmus policy entrepreneur fellow g...,F,19,professor
1,1,member aicpa wicpa brent graduated university ...,M,9,accountant
2,2,aster held teaching research positions ben gur...,M,19,professor
3,3,runs boutique design studio attending clients ...,M,24,architect
4,4,focuses cloud security identity access managem...,M,24,architect


In [16]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=28)

input_ids = []
attention_masks = []

for d in tqdm(df.description.values):
    bert_inp = bert_tokenizer.encode_plus(d,
                                      add_special_tokens = True,
                                      max_length =120,
                                      pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels = df['Category'].values

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/217197 [00:00<?, ?it/s]Truncation was not explicitl

In [17]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask = train_test_split(input_ids,
                                                                               labels,
                                                                               attention_masks,
                                                                               test_size=0.1,
                                                                              stratify=labels,
                                                                              random_state=1)

In [18]:
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bert_model.h5'

callbacks = [tensorflow.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),
            tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nBert Model',bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  21532     
Total params: 109,503,772
Trainable params: 109,503,772
Non-trainable params: 0
_________________________________________________________________

Bert Model None


In [19]:
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  21532     
Total params: 109,503,772
Trainable params: 109,503,772
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=bert_model.fit([train_inp,train_mask],
                       train_label,
                       batch_size=32,
                       epochs=4,validation_data=([val_inp,val_mask],val_label),
                       callbacks=callbacks)

Epoch 1/4
   1/6109 [..............................] - ETA: 2s - loss: 3.3037 - accuracy: 0.0312WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
6109/6109 [==============================] - 5202s 852ms/step - loss: 0.7021 - accuracy: 0.8020 - val_loss: 0.5673 - val_accuracy: 0.8318
Epoch 2/4
5042/6109 [=======================>......] - ETA: 14:20 - loss: 0.4688 - accuracy: 0.8592